# **Data Cleaning Notebook**

## Objectives

* Evaluate missing data
* Clean data


## Inputs

* outputs/datasets/collection/HousePrices.csv

## Outputs

* Generate cleaned Train and Test sets, both saved under outputs/datasets/cleaned


---

## Change working directory

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspaces/heritage-housing-issues/jupyter_notebooks'

Change the working directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspaces/heritage-housing-issues'

---

## Load Collected Data

In [4]:
import pandas as pd

df_raw_path = "outputs/datasets/collection/HousePrices.csv"
df = pd.read_csv(df_raw_path)
df.head(3)

1stFlrSF  2ndFlrSF  BedroomAbvGr BsmtExposure  BsmtFinSF1 BsmtFinType1  \
0       856     854.0           3.0           No         706          GLQ   
1      1262       0.0           3.0           Gd         978          ALQ   
2       920     866.0           3.0           Mn         486          GLQ   

   BsmtUnfSF  EnclosedPorch  GarageArea GarageFinish  ...  LotFrontage  \
0        150            0.0         548          RFn  ...         65.0   
1        284            NaN         460          RFn  ...         80.0   
2        434            0.0         608          RFn  ...         68.0   

   MasVnrArea OpenPorchSF  OverallCond  OverallQual  TotalBsmtSF  WoodDeckSF  \
0       196.0          61            5            7          856         0.0   
1         0.0           0            8            6         1262         NaN   
2       162.0          42            5            7          920         NaN   

   YearBuilt  YearRemodAdd  SalePrice  
0       2003          2003     208500  
1       1976          1976     181500  
2       2001          2002     223500  

[3 rows x 24 columns]

## Data Exploration

In [5]:
vars_with_missing_data = df.columns[df.isna().sum() > 0].to_list()
vars_with_missing_data

['2ndFlrSF',
 'BedroomAbvGr',
 'BsmtExposure',
 'BsmtFinType1',
 'EnclosedPorch',
 'GarageFinish',
 'GarageYrBlt',
 'LotFrontage',
 'MasVnrArea',
 'WoodDeckSF']

In [6]:
from ydata_profiling import ProfileReport

if vars_with_missing_data:
    profile = ProfileReport(df=df[vars_with_missing_data], minimal=True)
    profile.to_notebook_iframe()
else:
    print("No missing data found.")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

## Correlation and PPS Analysis

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps
import warnings

warnings.filterwarnings("ignore")

# Define heatmap functions
def heatmap_corr(df, threshold, figsize=(18, 12), font_annot=8):
    mask = np.zeros_like(df, dtype=bool)
    mask[np.triu_indices_from(mask)] = True
    mask[abs(df) < threshold] = True
    
    plt.figure(figsize=figsize)
    sns.heatmap(df, mask=mask, annot=True, cmap="viridis", annot_kws={"size": font_annot}, linewidths=0.5)
    plt.show()

def heatmap_pps(df, threshold, figsize=(18, 12), font_annot=8):
    mask = np.zeros_like(df, dtype=bool)
    mask[abs(df) < threshold] = True
    
    plt.figure(figsize=figsize)
    sns.heatmap(df, mask=mask, annot=True, cmap="rocket_r", annot_kws={"size": font_annot}, linewidths=0.5)
    plt.show()

def CalculateCorrAndPPS(df):
    df_corr_spearman = df.corr(method="spearman", numeric_only=True)
    df_corr_pearson = df.corr(method="pearson", numeric_only=True)

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.pivot(columns='x', index='y', values='ppscore')

    return df_corr_pearson, df_corr_spearman, pps_matrix

# Run analysis
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

# Display heatmaps
heatmap_corr(df_corr_pearson, threshold=0.4)
heatmap_corr(df_corr_spearman, threshold=0.4)
heatmap_pps(pps_matrix, threshold=0.2)

## Data Cleaning

### Evaluate Missing Data

In [10]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute / len(df) * 100, 2)
    df_missing_data = pd.DataFrame({
        "RowsWithMissingData": missing_data_absolute,
        "PercentageOfDataset": missing_data_percentage,
        "DataType": df.dtypes   
    }).sort_values(by="PercentageOfDataset", ascending=False).query("PercentageOfDataset > 0")
    return df_missing_data

# Run on full dataset
EvaluateMissingData(df)

RowsWithMissingData  PercentageOfDataset DataType
EnclosedPorch                 1324                90.68  float64
WoodDeckSF                    1305                89.38  float64
LotFrontage                    259                17.74  float64
GarageFinish                   235                16.10   object
BsmtFinType1                   145                 9.93   object
BedroomAbvGr                    99                 6.78  float64
2ndFlrSF                        86                 5.89  float64
GarageYrBlt                     81                 5.55  float64
BsmtExposure                    38                 2.60   object
MasVnrArea                       8                 0.55  float64

## Data Cleaning Strategy

In [11]:
drop_vars = ['BsmtExposure', 'GarageFinish', 'WoodDeckSF', 'KitchenQual', 'EnclosedPorch', 'LotFrontage', 'BsmtFinType1']

# Fill remaining missing numerical values (median)
fill_median_vars = ['GarageYrBlt', 'MasVnrArea', 'BedroomAbvGr']

# Fill remaining missing with Zero
fill_zero_vars = ['2ndFlrSF']

## Split Dataset into Train/Test 

In [12]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet = train_test_split(df, test_size=0.2, random_state=0)
print(f"TrainSet: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

TrainSet: (1168, 24) 
TestSet shape: (292, 24)


## Drop Variables

In [13]:
from feature_engine.selection import DropFeatures

dropper = DropFeatures(features_to_drop=drop_vars)
dropper.fit(TrainSet)
TrainSet = dropper.transform(TrainSet)
TestSet = dropper.transform(TestSet)
df = dropper.transform(df)

## Fill Zero Values

In [14]:
for col in fill_zero_vars:
    TrainSet[col].fillna(0, inplace=True)
    TestSet[col].fillna(0, inplace=True)
    df[col].fillna(0, inplace=True)

## Median Imputation

In [15]:
from sklearn.impute import SimpleImputer

median_imputer = SimpleImputer(strategy='median')
TrainSet[fill_median_vars] = median_imputer.fit_transform(TrainSet[fill_median_vars])
TestSet[fill_median_vars] = median_imputer.fit_transform(TestSet[fill_median_vars])
df[fill_median_vars] = median_imputer.fit_transform(df[fill_median_vars])

## Check Final Missing Values

In [17]:
print("Missing in TrainSet:")
print(TrainSet.isnull().sum())

print("\nMissing in TestSet:")
print(TestSet.isnull().sum())

print("\nMissing in full df:")
print(df.isnull().sum())


Missing in TrainSet:
1stFlrSF        0
2ndFlrSF        0
BedroomAbvGr    0
BsmtFinSF1      0
BsmtUnfSF       0
GarageArea      0
GarageYrBlt     0
GrLivArea       0
LotArea         0
MasVnrArea      0
OpenPorchSF     0
OverallCond     0
OverallQual     0
TotalBsmtSF     0
YearBuilt       0
YearRemodAdd    0
SalePrice       0
dtype: int64

Missing in TestSet:
1stFlrSF        0
2ndFlrSF        0
BedroomAbvGr    0
BsmtFinSF1      0
BsmtUnfSF       0
GarageArea      0
GarageYrBlt     0
GrLivArea       0
LotArea         0
MasVnrArea      0
OpenPorchSF     0
OverallCond     0
OverallQual     0
TotalBsmtSF     0
YearBuilt       0
YearRemodAdd    0
SalePrice       0
dtype: int64

Missing in full df:
1stFlrSF        0
2ndFlrSF        0
BedroomAbvGr    0
BsmtFinSF1      0
BsmtUnfSF       0
GarageArea      0
GarageYrBlt     0
GrLivArea       0
LotArea         0
MasVnrArea      0
OpenPorchSF     0
OverallCond     0
OverallQual     0
TotalBsmtSF     0
YearBuilt       0
YearRemodAdd    0
SalePrice  

In [18]:
print(f"TrainSet has {TrainSet.isnull().sum().sum()} missing values")

TrainSet has 0 missing values


In [ ]:
print(f"TestSet has {TestSet.isnull().sum().sum()} missing values")


TestSet has 0 missing values


In [20]:
print(f"DataSet has {df.isnull().sum().sum()} missing values")


DataSet has 0 missing values


---

# Push files to Repo

In [21]:
import os

try:
    os.makedirs(
        name="outputs/datasets/cleaned"
    )  # create outputs/datasets/cleaned folder
except Exception as e:
    print(e)

df.to_csv(f"outputs/datasets/cleaned/HousePricesCleaned.csv", index=False)
TrainSet.to_csv(f"outputs/datasets/cleaned/TrainSetCleaned.csv", index=False)
TestSet.to_csv(f"outputs/datasets/cleaned/TestSetCleaned.csv", index=False)


## Summary and the next steps

**Summary**

- Loaded raw data and inspected missing values.
- Dropped 7 low-quality or subjective features  
  (`BsmtExposure`, `GarageFinish`, `WoodDeckSF`, `KitchenQual`, `EnclosedPorch`, `LotFrontage`, `BsmtFinType1`).
- Imputed missing values:  
  - `2ndFlrSF` → filled with `0`  
  - `GarageYrBlt`, `MasVnrArea`, `BedroomAbvGr` → filled with `median`
- Split the dataset** into Train/Test sets.
- Confirmed that all missing values were handled.
- Saved cleaned datasets to `outputs/datasets/cleaned`.

---

**Next Steps**

- Create the Data Study Notebook.
- Analyze which features most influence `SalePrice`.
- Use visualizations such as scatter plots, box plots, and heatmaps.
